In [1]:
from env_function import environmental_variables

bbox = (-82.80, 27.30, -82.40, 27.70)  # Tampa Bay
start = "2024-11-01"
end = "2024-12-31"

env_data = environmental_variables(bbox, start, end, ["sst", "precip"])
sst = env_data.get("sst")
precip = env_data.get("precip")



Fetching SST from NOAA ERDDAP...
SST timesteps from ERDDAP: 1
SST final: 1 days

Searching for precip (NOAA MRMS)...
Found 1201 precip items
Precip: 1201 days


In [2]:
from env_function import environmental_variables

bbox = (-83.0, 27.3, -82.4, 28.0)  # slightly larger Tampa Bay box
start = "2024-01-01"
end   = "2024-03-01"

env_data = environmental_variables(bbox, start, end, ["sst", "precip"])
sst = env_data.get("sst")
precip = env_data.get("precip")



Fetching SST from NOAA ERDDAP...
SST timesteps from ERDDAP: 1
SST final: 1 days

Searching for precip (NOAA MRMS)...
Found 1464 precip items
Precip: 1464 days


In [3]:
sst = env_data.get("sst")
precip = env_data.get("precip")

print("\n=== SUCCESS SUMMARY ===")
print(f"SST shape: {sst.shape if sst is not None else 'None'}")
print(f"SST time range: {sst.time.values[0] if sst is not None and len(sst.time)>0 else 'N/A'}")
print(f"Precip shape: {precip.shape if precip is not None else 'None'}")
print(f"Precip time range: {precip.time.values[0]} to {precip.time.values[-1] if precip is not None else 'N/A'}")




=== SUCCESS SUMMARY ===
SST shape: (1, 1)
SST time range: 2025-12-03T12:00:00.000000000
Precip shape: (1464,)
Precip time range: 2024-01-01T00:00:00.000000000 to 2024-03-01T23:00:00.000000000


In [4]:
from env_function import environmental_variables

bbox = (-82.80, 27.30, -82.40, 27.70)  # Tampa Bay
start = "2019-01-01"  # Full historical range
end = "2024-12-31"

env_data = environmental_variables(bbox, start, end, ["sst", "precip"])

sst = env_data.get("sst")
precip = env_data.get("precip")

print("\n=== FINAL SUCCESS ===")
print(f"SST: {sst.time.size if sst is not None else 0} days ({sst.time.values[0]} to {sst.time.values[-1]})")
print(f"Precip: {len(precip.time) if precip is not None else 0} days")



Fetching SST from NOAA ERDDAP...
SST timesteps from ERDDAP: 1
SST final: 1 days

Searching for precip (NOAA MRMS)...
Found 2000 precip items
Precip: 2000 days

=== FINAL SUCCESS ===
SST: 1 days (2025-12-03T12:00:00.000000000 to 2025-12-03T12:00:00.000000000)
Precip: 2000 days
